**Transfer Learning** là kỹ thuật học máy là trong đó, một mô hình đã được xây dựng và huấn luyện sẵn để giải quyết một vấn đề nào đó, được sử dụng lại trong một vấn đề khác. Các mô hình này thường được huấn luyện với một bộ dữ liệu rất lớn trên những phần cứng mạnh mẽ và có độ chính xác, hiệu quả.

Một số pretrained model như là:


*   VGG (16, 19)
*   Resnet (18, 34, 50, 101, 152 ...)
*   Inception (v1, v2, v3, ...)
*   Xception 



### Transfer Learning với TF 2.0

Chúng ta sẽ tiến hành xây dựng hai mô hình phân loại chó và mèo, một mô hình sử dụng Transfer Learning và mô hình còn lại thì không 

In [153]:
! nvidia-smi

Tue Sep  1 10:02:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W /  70W |  14575MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [154]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [155]:
(train_raw, validation_raw, test_raw), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:20%]', 'train[80%:85%]', 'train[90%:95%]'],
    with_info=True,
    as_supervised=True,
    shuffle_files=True,
)

In [156]:
def processing(img, label):
  img = tf.cast(img, tf.float32)
  img = img/255.0
  img = tf.image.resize(img, (224, 224))
  return img, label

In [157]:
train = train_raw.map(processing)
validation = validation_raw.map(processing)
test = test_raw.map(processing)

In [158]:
BATCH_SIZE = 64            # batch size
BUFFER_SIZE = 1000          # suffle buffer size

In [159]:
train = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)
validation = validation.batch(BATCH_SIZE)

Chúng ta sẽ sử dụng mô hình Resnet 50, chúng ta sẽ dùng kiến trúc chứ không sử dụng trọng số của mô hình đã được huấn luyện sẵn.

In [160]:
input_shape = (224, 224, 3)

In [161]:
backbone1 = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights=None)
backbone1.trainable=True

In [ ]:
backbone1.summary()

In [170]:
flatten = tf.keras.layers.Flatten()
drop = tf.keras.layers.Dropout(0.25)
fc1 = tf.keras.layers.Dense(128, activation='relu')
fc2 = tf.keras.layers.Dense(1, activation="sigmoid")

In [171]:
model1 = tf.keras.Sequential([
    backbone1, 
    flatten,
    drop,
    fc2]
)

In [172]:
lr = 1e-5
model1.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics='acc')

epochs = 10
model1.fit(train, epochs=epochs, verbose=1, validation_data=validation)

Epoch 1/10
73/73 [==============================] - 55s 751ms/step - loss: 0.6949 - acc: 0.4968 - val_loss: 0.6925 - val_acc: 0.4815
Epoch 2/10
73/73 [==============================] - 55s 751ms/step - loss: 0.6888 - acc: 0.5140 - val_loss: 0.6929 - val_acc: 0.4815
Epoch 3/10
73/73 [==============================] - 55s 756ms/step - loss: 0.6848 - acc: 0.5322 - val_loss: 0.6932 - val_acc: 0.4815
Epoch 4/10
73/73 [==============================] - 56s 762ms/step - loss: 0.6790 - acc: 0.5552 - val_loss: 0.6920 - val_acc: 0.4824
Epoch 5/10
73/73 [==============================] - 56s 765ms/step - loss: 0.6761 - acc: 0.5572 - val_loss: 0.6927 - val_acc: 0.4832
Epoch 6/10
73/73 [==============================] - 56s 766ms/step - loss: 0.6708 - acc: 0.5817 - val_loss: 0.6888 - val_acc: 0.4910
Epoch 7/10
73/73 [==============================] - 56s 767ms/step - loss: 0.6670 - acc: 0.5933 - val_loss: 0.6852 - val_acc: 0.5099
Epoch 8/10
73/73 [==============================] - 56s 768ms/step - 

Chúng ta sẽ sử dụng mô hình đã được huấn luyện sẵn

In [110]:
backbone2 = tf.keras.applications.ResNet50V2(input_shape=input_shape, include_top=False, weights='imagenet')
backbone2.trainable=True

94674944/94668760 [==============================] - 1s 0us/step


In [114]:
drop = tf.keras.layers.Dropout(0.25)
flatten = tf.keras.layers.Flatten()
fc1 = tf.keras.layers.Dense(128, activation='relu')
fc2 = tf.keras.layers.Dense(1, activation="sigmoid")

In [115]:
model2 = tf.keras.Sequential([
    backbone2, 
    flatten,
    drop,
    fc1,
    drop,
    fc2]
)

In [116]:
lr = 1e-4
model2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics='acc')

epochs = 10
model2.fit(train, epochs=epochs, verbose=1, validation_data=validation)

Epoch 1/10
37/37 [==============================] - 47s 1s/step - loss: 0.5156 - acc: 0.9723 - val_loss: 0.5336 - val_acc: 0.9390
Epoch 2/10
37/37 [==============================] - 48s 1s/step - loss: 0.5089 - acc: 0.9862 - val_loss: 0.5254 - val_acc: 0.9518
Epoch 3/10
37/37 [==============================] - 47s 1s/step - loss: 0.5078 - acc: 0.9877 - val_loss: 0.5462 - val_acc: 0.9209
Epoch 4/10
31/37 [========================>.....] - ETA: 7s - loss: 0.5084 - acc: 0.9892

KeyboardInterrupt: ignored

In [105]:
epochs = 10
model2.fit(train, epochs=epochs, verbose=1, validation_data=validation)

Epoch 1/10


ResourceExhaustedError: ignored